# Loader

> This class loads the necessary files to analyze the polarization of the formated csv.

In [ ]:
#| default_exp Loader

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
from unidecode import unidecode
import regex as re

In [ ]:
#| export
class Loader:

    def __init__(self,
                 csv: str,    # Path to the csv file
                 virtue_dict: str,   # Path to the virtue dictionary
                 vice_dict: str      # Path to the vice dictionary
                 ):
        """Constructor for the Loader class"""
        self.csv = csv
        self.virtue_dict = self._load_dict('virtue', virtue_dict)
        self.vice_dict = self._load_dict('vice', vice_dict)
        self.original = None
        self.processed = None
        self.processed_bool = False

    def _load_dict(self, tag, path):
        """Loads a dictionary from a txt file"""
        dic = {}
        with open(path, 'r', encoding='utf-8') as f:
            words = [line.strip() for line in f]
        for word in words:
            dic[word] = tag
        return dic
    
    def process_csv(self,
                    text_cols:list, # List of text columns to process
                    num_cols:list, # List of numeric columns to process
                    date_cols:list # List of date columns to process
                    ):
        """Cleans the columns of the csv file prior the polarization analysis"""
        df = pd.read_csv(self.csv)
        self.original = df.copy()

        cols = text_cols + num_cols + date_cols
        df.drop_duplicates(keep='first',inplace=True) 

        def clean_message(text):
            if isinstance(text, str):
                # Eliminate urls
                regex_url = r'\b(?:https?|ftp):\/\/(?:www\.)?[-A-Z0-9+&@#\/%?=~_|!:,.;]*[-A-Z0-9+&@#\/%=~_|]'
                text = re.sub(regex_url, '', text)

                # accents and turn to lower case
                text = unidecode(text).lower()
                
                # Eliminate non-alphabetic and non-numeric characters
                text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

                # Eliminar emojis
                text = re.sub(r'\p{Emoticons}', '', text)

                # Eliminate additional spaces
                text = re.sub(r'\s+', ' ', text).strip()

            return text
        
        for col in text_cols:
            df[col] = df[col].apply(clean_message)
            df[col] = df[col].astype(str)

        for col in num_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')


        new = df.loc[:,cols]
        new = new.reset_index(drop=True)
        self.processed = new
        self.processed_bool = True
    
    def __str__(self) -> str:
        """Returns the string representation of the object"""
        return f'Loader(\n  csv={self.csv}\n  virtue_dict={self.virtue_dict.keys()}\n  vice_dict={self.vice_dict.keys()}\n)'

    __repr__ = __str__

    def set_virtue_dict(self,
                        virtue_dict: str # Path to the virtue dictionary
                        ):
        """Sets the virtue dictionary"""
        self.virtue_dict = self._load_dict('virtue', virtue_dict)
    
    def set_vice_dict(self,
                        virtue_dict: str # Path to the virtue dictionary
                        ):
        """Sets the virtue dictionary"""
        self.virtue_dict = self._load_dict('virtue', virtue_dict)

    def set_csv(self,
                csv: str # Path to the csv file
                ):
        """Sets the csv file"""
        self.csv = csv

    def set_processed(self,
                      processed: pd.DataFrame # Processed dataframe
                      ):
        """Sets the processed dataframe"""
        self.processed = processed
        self.processed_bool = True


In [ ]:
show_doc(Loader.process_csv)

---

### Loader.process_csv

>      Loader.process_csv (text_cols:list, num_cols:list, date_cols:list)

Cleans the columns of the csv file prior the polarization analysis

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| text_cols | list | List of text columns to process |
| num_cols | list | List of numeric columns to process |
| date_cols | list | List of date columns to process |

In [ ]:
show_doc(Loader.set_virtue_dict)

In [ ]:
show_doc(Loader.set_vice_dict)

In [ ]:
show_doc(Loader.set_csv)

In [ ]:
show_doc(Loader.set_processed)